## 1. Importar Librerías

Necesitaremos pandas para manejar los datos, google.generativeai para la generación, dotenv para cargar tu clave de API, y tqdm para ver el progreso.

In [1]:
import pandas as pd
import google.generativeai as genai
import os
import json
import time
from dotenv import load_dotenv
from tqdm.notebook import tqdm

print("Librerías importadas.")

Librerías importadas.


## 2. Cargar Clave de API y Configurar Gemini
Esta celda cargará el archivo .env que se creó para manejo de la API Key .

In [2]:
# Carga las variables del archivo .env
load_dotenv()

API_KEY = os.getenv('GOOGLE_API_KEY')

if not API_KEY:
    print("Error: No se encontró la GOOGLE_API_KEY.")
    print("Asegúrate de tener un archivo .env en esta carpeta con tu clave.")
else:
    genai.configure(api_key=API_KEY)
    print("API de Google Gemini configurada exitosamente.")

# Configura el modelo que usaremos para generar las preguntas
# Usamos Flash por su velocidad.
generation_model = genai.GenerativeModel('gemini-flash-latest')
print(f"Modelo seleccionado: {generation_model.model_name}")

API de Google Gemini configurada exitosamente.
Modelo seleccionado: models/gemini-flash-latest


## 3. Cargar el Gold Standard (Antiguo)

Cargamos el archivo JSON que ya tienes en un DataFrame de pandas para poder iterar sobre él fácilmente.

In [3]:
INPUT_FILE = 'gold_standard.json'
OUTPUT_FILE = 'gold_standard_conceptual.json' # El nuevo archivo que crearemos

try:
    df_gold = pd.read_json(INPUT_FILE)
    print(f"Gold Standard cargado exitosamente desde '{INPUT_FILE}'.")
    print(f"Se encontraron {len(df_gold)} preguntas.")
    df_gold.head()
except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{INPUT_FILE}' en esta carpeta.")
except Exception as e:
    print(f"Error al leer el JSON: {e}")

Gold Standard cargado exitosamente desde 'gold_standard.json'.
Se encontraron 108 preguntas.


## 4. Definir la Función de Generación por Lotes
Esta es la nueva función de generación. En lugar de procesar una fila, procesa un DataFrame (un lote) completo y construye un solo prompt gigante.

In [4]:
def generar_preguntas_conceptuales_en_lote(batch_df):
    """
    Usa Gemini para generar un lote de preguntas conceptuales en una sola solicitud.
    """
    
    # 1. Construir el prompt con todas las entradas del lote
    prompt_header = """
Tu tarea es generar un conjunto de preguntas de prueba para un sistema RAG.
Te daré una lista de "Entradas". Cada Entrada tiene una "Respuesta Ideal" y una "Pregunta Original".

Debes generar una "Pregunta Conceptual (Nueva)" para CADA Entrada de la lista.

REGLAS IMPORTANTES:
1. Genera una "Pregunta Conceptual" por cada "Entrada", en el mismo orden.
2. La pregunta debe ser semántica y NO usar las palabras clave de la "Pregunta Original".
3. **Formato de Salida Obligatorio:** Devuelve *solo* la lista de preguntas nuevas, una por línea.
   No añadas números, comillas, ni texto introductorio como "Aquí están las preguntas:".

--- INICIO DE ENTRADAS ---
"""
    
    entradas_prompt = []
    for index, row in batch_df.iterrows():
        entrada = f"""
Entrada {index + 1}:
Respuesta Ideal: "{row['ideal_answer']}"
Pregunta Original: "{row['query']}"
"""
        entradas_prompt.append(entrada)
    
    prompt_footer = "\n--- INICIO DE SALIDA (SOLO PREGUNTAS NUEVAS, UNA POR LÍNEA) ---"
    
    # Prompt final
    prompt_completo = prompt_header + "\n".join(entradas_prompt) + prompt_footer
    
    # 2. Llamar a la API (con reintentos)
    try:
        for _ in range(3): # Reintenta 3 veces
            try:
                response = generation_model.generate_content(prompt_completo)
                # El resultado es un solo string con saltos de línea
                preguntas_generadas = response.text.strip().split('\n')
                
                # Verificación: ¿Coincide el número de preguntas?
                if len(preguntas_generadas) == len(batch_df):
                    return [q.strip().replace('"', '') for q in preguntas_generadas]
                else:
                    print(f"Error de formato: Se esperaban {len(batch_df)} preguntas, pero se recibieron {len(preguntas_generadas)}.")
                    print(f"Respuesta recibida: {response.text}")

            except Exception as e:
                print(f"Advertencia: Error de API ({e}), reintentando en 3 segundos...")
                time.sleep(3)
        
        # Si todos los reintentos fallan
        print(f"Error: Fallaron todos los reintentos para un lote de {len(batch_df)} preguntas.")
        return [f"ERROR_AL_GENERAR_PREGUNTA" for _ in range(len(batch_df))]

    except Exception as e:
        print(f"Error fatal generando lote: {e}")
        return [f"ERROR_FATAL" for _ in range(len(batch_df))]

# Probar la función una vez con un lote de 2
print("--- Probando la función de generación por lotes ---")
df_prueba_lote = df_gold.head(2)
preguntas_lote = generar_preguntas_conceptuales_en_lote(df_prueba_lote)

print("PREGUNTAS CONCEPTUALES GENERADAS (Lote):")
print(preguntas_lote)

--- Probando la función de generación por lotes ---
PREGUNTAS CONCEPTUALES GENERADAS (Lote):
['¿Qué elementos estructurales componen la arquitectura de justicia transicional y cuál fue la extensión temporal inicial del organismo de esclarecimiento de la verdad?', '¿Cuál fue la gran interrogante que guio el trabajo del ente de la verdad y qué segmentos de la población civil fueron señalados por su corresponsabilidad en la persistencia de la violencia?']


## 5: Ejecutar la Generación por Lotes (Respetando Límite de 10 RPM)
Esta celda reemplaza el progress_apply. Crea los lotes, llama a la función de la Celda 4, y espera 6 segundos después de cada llamada para respetar el límite de 10 RPM (10 solicitudes * 6 seg = 60 seg).

In [5]:
# Tamaño del lote: 10 preguntas por solicitud
# (10 solicitudes / minuto = 1 solicitud cada 6 segundos)
BATCH_SIZE = 10
SEGUNDOS_ENTRE_SOLICITUDES = 6 # (60 segundos / 10 solicitudes)

# Lista para guardar todas las nuevas preguntas en orden
lista_nuevas_preguntas = []

print(f"Iniciando generación de {len(df_gold)} preguntas en lotes de {BATCH_SIZE}...")
print(f"Esto tomará aprox. { (len(df_gold) / BATCH_SIZE) * SEGUNDOS_ENTRE_SOLICITUDES / 60 :.2f} minutos.")

# Usamos tqdm en el rango para la barra de progreso
for i in tqdm(range(0, len(df_gold), BATCH_SIZE), desc="Procesando Lotes"):
    
    # 1. Obtener el lote
    df_batch = df_gold.iloc[i : i + BATCH_SIZE]
    
    # 2. Generar las preguntas para este lote
    nuevas_preguntas_lote = generar_preguntas_conceptuales_en_lote(df_batch)
    
    # 3. Guardar los resultados
    lista_nuevas_preguntas.extend(nuevas_preguntas_lote)
    
    # 4. Respetar el límite de la API
    # (No esperamos *después* de la última solicitud)
    if i + BATCH_SIZE < len(df_gold):
        time.sleep(SEGUNDOS_ENTRE_SOLICITUDES)

print("\n--- Generación completada ---")

# 5. Asignar las nuevas preguntas al DataFrame
df_gold['query_conceptual'] = lista_nuevas_preguntas

print("Vista previa de los resultados:")
df_gold.head()

Iniciando generación de 108 preguntas en lotes de 10...
Esto tomará aprox. 1.08 minutos.


Procesando Lotes:   0%|          | 0/11 [00:00<?, ?it/s]


--- Generación completada ---
Vista previa de los resultados:


,query,relevant_chunk_ids,ideal_answer,source_chapter,query_conceptual
0,¿Cuáles son los tres pilares del Sistema Integ...,"[01-prólogo.txt_0003, 01-prólogo.txt_0004]",Los tres pilares del Sistema Integral para la ...,01-prólogo.txt,¿Cómo se articula el mecanismo de justicia tra...
1,¿Cuál fue la pregunta macro que orientó la inv...,"[01-prólogo.txt_0013, 01-prólogo.txt_0014, 01-...","La pregunta macro fue: ¿por qué, a pesar de lo...",01-prólogo.txt,¿Cuál fue el interrogante central que guio el ...
2,¿De cuántos tomos y qué tipo de documento comp...,[01-prólogo.txt_0019],El informe final consta de diez tomos y una de...,01-prólogo.txt,Describa la composición estructural del docume...
3,¿Qué tipos de información y variables específi...,[],La Comisión de la Verdad utilizó datos del SIM...,02-introducción.txt,¿Qué clases de métricas cuantitativas sobre te...
4,Identifique las 'grietas o fallas geológicas' ...,"[02-introducción.txt_0009, 02-introducción.txt...",Los ensayistas de la CHCV mencionaron la cuest...,02-introducción.txt,¿Cuáles fueron las debilidades históricas fund...


## 6. Revisar y Guardar el Nuevo Gold Standard
Finalmente, guardamos nuestros resultados en un nuevo archivo JSON. Nos aseguramos de mantener las columnas importantes (relevant_chunk_ids, ideal_answer) y usamos nuestra nueva query_conceptual como la query principal.

In [6]:
# Creamos una copia para no modificar el original en memoria
df_nuevo_gold = df_gold.copy()

# Renombramos nuestra nueva columna al nombre 'query' estándar
df_nuevo_gold = df_nuevo_gold.rename(columns={'query_conceptual': 'query'})

# Seleccionamos las columnas que necesita nuestro evaluador
columnas_finales = ['query', 'relevant_chunk_ids', 'ideal_answer', 'source_chapter']
df_nuevo_gold = df_nuevo_gold[columnas_finales]

# Revisamos las primeras 5 filas del nuevo Gold Standard
print("--- Vista previa del NUEVO Gold Standard (Conceptual) ---")
print(df_nuevo_gold.head().to_markdown(index=False))

# Guardar como JSON
try:
    # Convertimos el DataFrame a una lista de diccionarios
    json_records = df_nuevo_gold.to_dict('records')
    
    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(json_records, f, ensure_ascii=False, indent=4)
        
    print(f"\n¡Éxito! Nuevo Gold Standard guardado en: '{OUTPUT_FILE}'")
    print("Este archivo ahora contiene las preguntas conceptuales.")

except Exception as e:
    print(f"Error al guardar el archivo JSON: {e}")

--- Vista previa del NUEVO Gold Standard (Conceptual) ---
| query                                                                                                                                                                                                                                            | query                                                                                                                                                                                                                                  | relevant_chunk_ids                                                                           | ideal_answer                                                                                                                                                                                                                                                                                                                                                                     

C:\Users\user\AppData\Local\Temp\ipykernel_78192\4161709486.py:18: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  json_records = df_nuevo_gold.to_dict('records')
